# Feature selection


## Dependencies


The dependencies used are as follows


In [1]:
from sklearn.model_selection import cross_val_score

## Selection methods


After hyperparameter tuning, we proceed to attribute selection. Three methods will be used for this

- To see the most important attributes, PermutationImportance
- For a more exhaustive search, SequentialForwardSelector
- For a more stochastic search, GeneticAlgorithms

The main method will be Sequential forward selection (SFS), in which features are sequentially added to an empty candidate set until the addition of more features does not lower the criterion.

PermutationImportance we will use it to corroborate the results, as well as to see which attributes contribute the most to the performance of the model. The performance obtained by this measure and the previous one may differ because, even if one measure is of little relevance by itself, combined with others it can improve the model significantly.

Finally, we will use genetic algorithms to check with a small stochastic search if there is a possibility that there are other combinations that improve performance. This is because SequentialForwardSelector adds measures starting from one measure, i.e., it does not check all combinations and there may be a better one. Regarding the genetic algorithm itself, the fitness function will correspond to the cross-validation of a binary individual, where a 1 in position i will represent that measure i is taken for the evaluation, and if it is 0 it is not.


In [2]:
def fitness_func(ga_instance, individual, individual_idx):
    res = []

    if rank:
        individual[X.columns.get_loc("qid")] = 1

    idx = [i for i in range(len(individual)) if individual[i] == 1]

    attributes = X.iloc[:, idx]
    objective = y

    if not attributes.empty:
        res.extend(
            cross_val_score(
                estimator=estimator,
                X=attributes,
                y=objective,
                cv=tscv,
                scoring=scor,
                n_jobs=-1,
            )
        )

    avg_cross_val_score = sum(res) / len(res) if not attributes.empty else -10000
    return avg_cross_val_score